In [24]:
import pandas as pd
import numpy as np
import requests
from requests import Request
import json
import csv
import dotenv
import os
from dotenv import load_dotenv, find_dotenv
from flipside import Flipside
import logging
# from degate.spot import Spot as Client
# from degate.lib.utils import config_logging
from concurrent.futures import ThreadPoolExecutor, as_completed

In [15]:
load_dotenv()
dotenv.load_dotenv('../../.env')
api_key = os.environ["DUNE_API_KEY"]
headers = {"X-Dune-API-Key": api_key}
flipside_key = os.environ["FLIPSIDE_API_KEY"]

In [ ]:
#28/02 CSV file with token metadata was created by datege_fetch_tokens_metadata.ipynb script

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Load token list from CSV
try:
    df = pd.read_csv("token_metadata.csv").rename(columns={'tokenid': 'id'}).drop(columns=['__row_index'], errors='ignore')
except FileNotFoundError:
    logging.error("File 'token_metadata.csv' not found. Exiting.")
    exit(1)

# Create a dictionary for token lookup (symbol -> id)
token_dict = df.set_index("symbol")["id"].to_dict()

# Define valid trading pairs for each quote currency
QUOTE_CURRENCY_PAIRS = {
    "USDT": "all",
    "USDC": "all",
    "WBTC": ["ETH", "SOL"],
    "ETH": "all",
    "USDM": ["ETH", "DG", "MAP", "wsETH", "WBTC", "GRT"],
    "LUSD": ["ETH"]
}

# Base URL for API requests
#https://api-docs.degate.com/spot/#symbol-order-book-ticker
BASE_URL = "https://v1-mainnet-backend.degate.com/order-book-ws-api/ticker/bookTicker"

# Filter out test tokens early to avoid unnecessary processing
filtered_tokens = {symbol: token_id for symbol, token_id in token_dict.items() if "TEST" not in symbol}

# Use a session for connection reuse
session = requests.Session() 

def fetch_ticker(base_symbol, base_id, quote_symbol, quote_id):
    """
    Fetch ticker data for a given base-quote pair and return structured data if valid.
    """
    pair_name = f"{base_symbol}/{quote_symbol}"
    params = {"base_token_id": base_id, "quote_token_id": quote_id}
    
    try:
        response = session.get(BASE_URL, params=params, timeout=5)
        response.raise_for_status()
        result = response.json()

        # Validate API response
        if not isinstance(result, dict) or "code" not in result:
            logging.error(f"Unexpected response format for {pair_name}: {result}")
            return None

        if result.get("code") == 0:
            logging.info(f"Valid trading pair found: {pair_name}")
            return {
                "pair": pair_name,
                "base_symbol": base_symbol,
                "base_id": base_id,
                "quote_symbol": quote_symbol,
                "quote_id": quote_id
            }
    except requests.RequestException as e:
        logging.error(f"Error fetching data for {pair_name}: {e}")
    
    return None

def main():
    """
    Main function to fetch and validate trading pairs.
    """()
    valid_trading_pairs = []
    
    # Prepare a list of valid quote tokens to avoid unnecessary iterations
    valid_quote_tokens = {q for q in QUOTE_CURRENCY_PAIRS.keys() if q in token_dict}

    # ThreadPoolExecutor with dynamic task submission
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = []

        for base_symbol, base_id in filtered_tokens.items():
            for quote_symbol in valid_quote_tokens:
                valid_bases = QUOTE_CURRENCY_PAIRS[quote_symbol]
                
                # Skip if quote symbol doesn't allow trading with base
                if valid_bases != "all" and base_symbol not in valid_bases:
                    continue

                quote_id = token_dict[quote_symbol]
                futures.append(executor.submit(fetch_ticker, base_symbol, base_id, quote_symbol, quote_id))

                # Prevent excessive task queuing
                if len(futures) >= 500:  # Process task in batches (500 at a time) to prevent memory overload 
                    for future in as_completed(futures): # as_completed returns futures in order of completion
                        result = future.result()
                        if result:
                            valid_trading_pairs.append(result)
                    futures.clear() # free up memory before next batch

        # Process remaining tasks
        for future in as_completed(futures):
            result = future.result()
            if result:
                valid_trading_pairs.append(result)

    # Save valid pairs to CSV with 5 columns
    if valid_trading_pairs:
        df_valid_pairs = pd.DataFrame(valid_trading_pairs)
        df_valid_pairs.to_csv("valid_trading_pairs.csv", index=False)
        logging.info("Valid trading pairs saved successfully.")
    else:
        logging.warning("No valid trading pairs found.")

if __name__ == "__main__":
    main()

# 01/03 I have enother main() function below to fetch trading data from Degate API 
# do i need separate main() functions for each script?


